In [ ]:
#This updates the injury test file: https://docs.google.com/spreadsheets/d/19lkC0kjt52UhysvWCu17YCz8CzZajQ4FzsZ3L8zeZcc/edit

In [10]:
#Packages
#Time
import time
from datetime import datetime, timedelta, timezone
from datetime import date
timestart = datetime.now(timezone(timedelta(hours=-4), 'EST'))
#Pandas
import pandas as pd
#Gspread
import gspread
import gspread_dataframe as gd
from oauth2client.service_account import ServiceAccountCredentials
#Beatiful Soup
from bs4 import BeautifulSoup
import bs4 as bs
#UrlLib
from urllib.request import Request, urlopen
import urllib.request, json 

In [2]:
#defining header
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

In [3]:
#Let's get the Slate Data
#Let's Open NBASlateFeedBuild
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope) 
gc = gspread.authorize(credentials)
sheet = gc.open('InjuryTest')
w1 = sheet.worksheet("Control")

nfllink = w1.acell('b1').value
nbalink = w1.acell('b2').value

req = urllib.request.Request(url=nfllink, headers=header)
with urllib.request.urlopen(req) as url:
    datanfl = json.loads(url.read().decode())

    
req = urllib.request.Request(url=nbalink, headers=header)
with urllib.request.urlopen(req) as url:
    datanba = json.loads(url.read().decode())

In [4]:
datatable=[] 
for players in datanfl:
    PlayerID = players['PlayerID']
    Player = players['Name']
    Position = players['Position']
    Team = players['Team']
    Inj = players['InjuryBodyPart']
    Injn =players['InjuryNotes']
    InjS =players['InjuryStatus']
    DepthPos = players['DepthPosition']
    DepthOrder = players['DepthOrder']
    DepthCat = players['DepthPositionCategory']
    Inactive = players['DeclaredInactive']
    
    row=[PlayerID,Player,Position,Team,Inj,Injn,InjS,DepthPos,DepthOrder, DepthCat, Inactive]
    datatable.append(row)

In [5]:
df=pd.DataFrame(datatable)
df.columns = ["PlayerID","Player","Position","Team","Inj","Inj Notes", "Inj Status" ,"DepthPos","DepthOrder", "DepthCat", "Inactive"]
df

,PlayerID,Player,Position,Team,Inj,Inj Notes,Inj Status,DepthPos,DepthOrder,DepthCat,Inactive
0,22836,Justin Hilliard,OLB,None,Suspension,Hilliard has been suspended two games for viol...,Out,None,NaN,None,True
1,549,Matt Prater,K,ARI,Hip,Prater was limited in practice on Thursday hea...,Questionable,PK,1.0,ST,False
2,13037,Rodney Hudson,OL,ARI,Knee,Hudson did not practice on Thursday heading in...,Questionable,C,1.0,OFF,False
3,15094,Justin Pugh,OL,ARI,Knee (ACL),Pugh has suffered a torn ACL and will miss the...,Out,None,NaN,None,True
4,16785,D.J. Humphries,OL,ARI,Back,Humphries did not practice on Thursday heading...,Questionable,LT,1.0,OFF,False
...,...,...,...,...,...,...,...,...,...,...,...
517,23072,Milo Eifler,LB,WAS,Hamstring,Eifler has been placed on the injured reserve ...,Out,None,NaN,None,True
518,23155,Jahan Dotson,WR,WAS,Hamstring,Dotson has been ruled out for Week 8 versus th...,Out,RWR,1.0,OFF,True
519,23276,Phidarian Mathis,DT,WAS,Torn Meniscus,Mathis has been placed on the injured reserve ...,Out,None,NaN,None,True
520,23488,Curtis Hodges,TE,WAS,Thigh,Hodges has been placed on Injured Reserve with...,Out,None,NaN,None,True


In [6]:
datatable=[]

for players in datanba:
    PlayerID = players['PlayerID']
    Player = players['FirstName']+' '+players['LastName']
    Position = players['Position']
    Team = players['Team']
    try:
        Inj = players['InjuryBodyPart']
        Injn =players['InjuryNotes']
        InjS =players['InjuryStatus']
    except:
        Inj = ''
        Injn =''
        InjS =''
        
    DepthPos = players['DepthChartPosition']
    DepthOrder = players['DepthChartOrder']
    if Inj is None:
        blank=''
    else:
        row=[PlayerID,Player,Position,Team,Inj,Injn,InjS,DepthPos,DepthOrder]
        datatable.append(row)

In [7]:
df2=pd.DataFrame(datatable)
df2.columns = ["PlayerID","Player","Position","Team","Inj","Inj Notes", "Inj Status" ,"DepthPos","DepthOrder"]
df2

,PlayerID,Player,Position,Team,Inj,Inj Notes,Inj Status,DepthPos,DepthOrder
0,20000443,Otto Porter Jr.,SF,TOR,Hamstring,Porter Jr. is dealing with a hamstring injury ...,Out,SF,2.0
1,20000468,Anthony Davis,PF,LAL,Back,Davis is dealing with a back injury and is unc...,Questionable,C,1.0
2,20000487,Andre Iguodala,SF,GS,Injury Management,Iguodala is conditioning for game readiness an...,Out,SF,2.0
3,20000500,Khris Middleton,SF,MIL,Wrist,Middleton is rehabbing from a wrist injury and...,Out,SF,1.0
4,20000562,Jae Crowder,PF,PHO,Not Injury Related,Crowder is not with the team and will not take...,Out,None,NaN
...,...,...,...,...,...,...,...,...,...
81,20002889,Walker Kessler,C,UTA,Illness,Kessler is dealing with an illness and is unce...,Questionable,C,2.0
82,20002896,Jalen Williams,SF,OKC,Ankle,Williams is dealing with a right ankle effusio...,Out,SF,2.0
83,20002936,Cole Swider,PF,LAL,Foot,Swider is battling foot soreness and will not ...,Out,None,NaN
84,20002937,Collin Gillespie,PG,DEN,Lower Leg,Gillespie is recovering from surgery to repair...,Out,None,NaN


In [8]:
#Let's push props
Goal = sheet.worksheet("NFL")
sheet.values_clear("NFL!a1:k1000")
gd.set_with_dataframe(Goal, df, row=1,col=1)
timenow = datetime.now(timezone(timedelta(hours=-4), 'EST'))
w1.update('c1', timenow.strftime("%B %d, %Y %H:%M:%S"))

{'spreadsheetId': '19lkC0kjt52UhysvWCu17YCz8CzZajQ4FzsZ3L8zeZcc',
 'updatedRange': 'Control!C1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [9]:
#Let's push props
Goal = sheet.worksheet("NBA")
sheet.values_clear("NBA!a1:z1000")
gd.set_with_dataframe(Goal, df2, row=1,col=1)
timenow = datetime.now(timezone(timedelta(hours=-4), 'EST'))
w1.update('c2', timenow.strftime("%B %d, %Y %H:%M:%S"))

{'spreadsheetId': '19lkC0kjt52UhysvWCu17YCz8CzZajQ4FzsZ3L8zeZcc',
 'updatedRange': 'Control!C2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}